## Lesson 2: Connecting with a CRM

## Preparation 
<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> and other files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

In [1]:
# Before you start, please run the following code to set up your environment.
# This code will reset the environment (if needed) and prepare the resources for the lesson.
# It does this by quickly running through all the code from the previous lessons.

!sh ./ro_shared_data/reset.sh
%run ./ro_shared_data/lesson_2_prep.py lesson2

import os

agentId = os.environ['BEDROCK_AGENT_ID']
agentAliasId = os.environ['BEDROCK_AGENT_ALIAS_ID']
region_name = 'us-west-2'
lambda_function_arn = os.environ['LAMBDA_FUNCTION_ARN']

Resetting environment (if nessesary)
Agent reset process completed.
Lambda reset process completed.
Guardrail reset process completed.
Environment reset complete.
Lesson 2 Prep
Waiting for agent status of 'NOT_PREPARED'...
Agent status: CREATING
Agent status: NOT_PREPARED
Agent reached 'NOT_PREPARED' status.
Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Agent reached 'PREPARED' status.
Waiting for agent alias status of 'PREPARED'...
Agent alias status: CREATING
Agent alias status: CREATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


## Start of lesson

In [4]:
import boto3
import uuid
from helper import *

In [5]:
sessionId = str(uuid.uuid4())
message = "My name is Mike, my mug is broken and I want a refund."

In [6]:
invoke_agent_and_print(
    agentId=agentId, 
    agentAliasId=agentAliasId, 
    inputText=message, 
    sessionId=sessionId
)

User: My name is Mike, my mug is broken and I want a refund.

Agent: I apologize, but I do not have the necessary information to process
       your refund request for the broken mug. As a front-line
       customer support agent, I am limited in the actions I can take
       directly. The best thing for me to do is route your request to
       a human customer support agent who can properly assist you.
       They will be able to look up the details of your order and
       determine the appropriate next steps. Please hold while I
       transfer you to an agent who can help.

Session ID: a17a2290-28b5-444e-a561-01b6d83246cf


<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; The file that is examined in the video is at  <code>./ro_shared_data/functions/lambda_stage_1.py</code> </p>

Inside Amazon Bedrock, Action and Action Groups is what we need to connect our agent to different tools in order to give more knowledge response to our agents. The logic to connet for these actions are lambda functions.

In [7]:
bedrock_agent = boto3.client(service_name = 'bedrock-agent', region_name = region_name)

In [8]:
create_agent_action_group_response = bedrock_agent.create_agent_action_group(
    actionGroupName='customer-support-actions',
    agentId=agentId,
    actionGroupExecutor={
        'lambda': lambda_function_arn
    },
    functionSchema={
        'functions': [
            {
                'name': 'customerId',
                'description': 'Get a customer ID given available details. At least one parameter must be sent to the function. This is private information and must not be given to the user.', # what the function does
                'parameters': {
                    'email': {
                        'description': 'Email address',
                        'required': False,
                        'type': 'string'
                    },
                    'name': {
                        'description': 'Customer name',
                        'required': False,
                        'type': 'string'
                    },
                    'phone': {
                        'description': 'Phone number',
                        'required': False,
                        'type': 'string'
                    },
                }
            },            
            {
                'name': 'sendToSupport', # if agent can't help, message for human in the loop
                'description': 'Send a message to the support team, used for service escalation. ',
                'parameters': {
                    'custId': {
                        'description': 'customer ID',
                        'required': True,
                        'type': 'string'
                    },
                    'supportSummary': {
                        'description': 'Summary of the support request',
                        'required': True,
                        'type': 'string'
                    }
                }
            }
        ]
    },
    agentVersion='DRAFT',
)

In [9]:
create_agent_action_group_response

{'ResponseMetadata': {'RequestId': '4b8fe304-cc02-4cfd-b0c5-120c0cacbcd0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 13 Feb 2025 11:47:59 GMT',
   'content-type': 'application/json',
   'content-length': '1187',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4b8fe304-cc02-4cfd-b0c5-120c0cacbcd0',
   'x-amz-apigw-id': 'F69NfHc9PHcEXgw=',
   'x-amzn-trace-id': 'Root=1-67addbef-2b38aa836d75859725b3b20f'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-west-2:058264417278:function:dlai-support-agent-L4XBM'},
  'actionGroupId': 'ITAKUD1IVI',
  'actionGroupName': 'customer-support-actions',
  'actionGroupState': 'ENABLED',
  'agentId': 'YVSAFI40UB',
  'agentVersion': 'DRAFT',
  'createdAt': datetime.datetime(2025, 2, 13, 11, 47, 59, 656325, tzinfo=tzlocal()),
  'functionSchema': {'functions': [{'description': 'Get a customer ID given available details. At least one parameter must be sent to the function. This is privat

In [10]:
actionGroupId = create_agent_action_group_response['agentActionGroup']['actionGroupId']


In [11]:
wait_for_action_group_status(
    agentId=agentId, 
    actionGroupId=actionGroupId,
    targetStatus='ENABLED'
)

Action Group status: ENABLED


'ENABLED'

Since we performed different changes to the agent we need to prepare the agent again

In [12]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

wait_for_agent_status(
    agentId=agentId,
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Agent reached 'PREPARED' status.


In [13]:
bedrock_agent.update_agent_alias(
    agentId=agentId,
    agentAliasId=agentAliasId,
    agentAliasName='MyAgentAlias',
)

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

Waiting for agent alias status of 'PREPARED'...
Agent alias status: UPDATING
Agent alias status: UPDATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


### Now use the agent with functions

In [15]:
sessionId = str(uuid.uuid4())
message = "My name is Mike (mike@mike.com), my mug is broken and I want a refund."

In [14]:
invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=False
)

User: My name is Mike, my mug is broken and I want a refund.

Agent: I apologize, but I don't have enough details to fully process your
       request for a refund on your broken mug. Could you please
       provide any additional information about the purchase, such as
       when and where you bought the mug, or any order or receipt
       numbers? With those details, I can better assist you in getting
       this resolved. Please let me know if you have any other
       questions.

Session ID: a17a2290-28b5-444e-a561-01b6d83246cf


In [15]:
invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=True
)

User: My name is Mike, my mug is broken and I want a refund.

Agent: 
Agent's thought process:
  I already have the customer ID for Mike, so I can try sending the
  support request again with the additional details provided.

Invocation Input:
  Type: ACTION_GROUP
  Action Group: customer-support-actions
  Function: sendToSupport
  Parameters: [{'name': 'supportSummary', 'type': 'string', 'value': "Mike's mug is broken and he wants a refund."}, {'name': 'custId', 'type': 'string', 'value': '8934'}]

Observation:
  Type: ACTION_GROUP
  Action Group Output: {'error':'Details missing.'}

Agent's thought process:
  The sendToSupport function is still indicating that details are
  missing, so I should ask the user for more information before
  routing the request.

Observation:
  Type: FINISH

Final response:
  I apologize, but I still don't have enough details to fully process
  your request for a refund on your broken mug. Could you please
  provide any additional information you have, su